# Twitter API v1.1 Client

In [1]:
import tweepy
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

## Autentication

In [2]:
def get_auth_twitter():
        
    consumer_key= 'drhztkGNhICDt3cwBGXi7zzQk'
    consumer_secret= 'FDeCBlSi4qIbR9oPsf3tphQjCO6HqZfYANWceYIdJDZHsOaabi'
    access_token= '207778119-E9W1kqRXBbNeU2n8vcA0OoMn9XZpE9yvmZrD8gUx'
    access_token_secret= 'dncbYQQrh5jlJ97OmgxeaGgqOFiC7jnNpXdPngWMCRtDE'
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

    return api


## Query API V1

In [3]:
def search_for_tweets(api, search_words, until, lang, items):

        ##search_words = "DisneyPlus"
        ##until= "2020-11-03"
        ##lang = 'pt'

        cursor = tweepy.Cursor(api.search,
                    q=search_words,
                    lang=lang,
                    until= until, count=100).items(items)

        return cursor

## Cursor to List

In [4]:
def iterator_to_list(cursor):

    return list(cursor)      

## Creating a DF

In [5]:

def tweets_to_dataframe(tweets_list):

    columns = set()
    
    tweets_data = []

    
    for tweet in tweets_list:
        keys = tweet._json.keys()
        single_data = {}
        
        
        for k in keys:


            if tweet._json.get(k) == None:

                try:

                    single_data[k] = 0

                except Exception as e:
                    print(e)
                
            
            else:
              
               try:
                    single_data[k] = tweet._json.get(k)
               
               except Exception as e:
                    print(e)
                    single_data[k] = 0

                
            
            columns.add(k)

        tweets_data.append(single_data)    
    

    header_cols = list(columns)
    df = pd.DataFrame(tweets_data, columns=header_cols)
    df.head()
    
    return df

## Extracting data from objects

Entities column: finding mentions

In [6]:
def mentions(df):

        df['screen_name_mention_1'] = df['entities'].apply(lambda x: x['user_mentions'][0]['screen_name'] if x['user_mentions'] else None)

        df['id_mention_1'] = df['entities'].apply(lambda x: x['user_mentions'][0]['id_str'] if x['user_mentions'] else 0)

        return df

Retweeted_Status - Finding the author of the original tweet

In [7]:
def retweets(df):

    df['retweeted_from_screen_name'] = df['retweeted_status'].apply(lambda x: x['user']['screen_name'] if x is not np.nan else None)

    df['retweeted_from_id'] = df['retweeted_status'].apply(lambda x: x['user']['id_str'] if x is not np.nan else 0)

    df['retweeted_from_is_verified'] = df['retweeted_status'].apply(lambda x: x['user']['verified'] if x is not np.nan else None)

    return df 

User - Information about the user

In [8]:
def user(df):

    df['user_screen_name'] = df['user'].apply(lambda x: x['screen_name'] if x is not np.nan else None)

    df['user_id'] = df['user'].apply(lambda x: x['id_str'] if x is not np.nan else 0)

    df['user_followers'] = df['user'].apply(lambda x: x['followers_count'] if x is not np.nan else None)

    df['user_following'] = df['user'].apply(lambda x: x['friends_count'] if x is not np.nan else None)

    df['user_is_verified'] = df['user'].apply(lambda x: x['verified'] if x is not np.nan else None)

    return df

In [9]:
## New Columns

def destrinchando_colunas(df):
    
    mentions(df)
    retweets(df)
    user(df)

    return df


## Formating Datetime

In [10]:
def format_created_at(df):

    df['created_at'] = df['created_at'].apply(lambda x: datetime.strptime(x,'%a %b %d %H:%M:%S +0000 %Y'))

    return df

## Creating files

In [11]:
def create_csv(df, file_name):
    
    df.to_csv(f'C:/Users/amand/Documents/Jupyter Notebooks/DisneyPlus/data_final_csv/{file_name}.csv')

In [12]:
def create_pickle(df, file_name):

    df.to_pickle(f'C:/Users/amand/Documents/Jupyter Notebooks/DisneyPlus/data_final_pkl/{file_name}.pkl')

## Columns Filter

In [13]:
def filter_columns(df):

    important_columns = ['user_screen_name', 'user_id', 'user_is_verified', 'retweeted_from_screen_name', 'retweeted_from_id', 'retweeted_from_is_verified', 'screen_name_mention_1' , 'id_mention_1', 'in_reply_to_screen_name', 'in_reply_to_user_id', 'created_at', 'id', 'text', 'retweet_count', 'favorite_count', 'user_followers','user_following']

    df = df[important_columns]

    return df


In [14]:
def filter_columns_graph_ids(df):

    graph_columns_ids = ['user_id', 'retweeted_from_id' , 'id_mention_1', 'in_reply_to_user_id',  'id']

    df = df[graph_columns_ids]

    return df

## Todo o processo

In [15]:
def pega_tweets(search_words, until, lang, items, file_name):

    api = get_auth_twitter()

    cursor = search_for_tweets(api, search_words, until, lang, items)

    tweets_list = iterator_to_list(cursor)

    df = tweets_to_dataframe(tweets_list)
    df = destrinchando_colunas(df)
    df = format_created_at(df)

    create_csv(df, file_name)
    create_pickle(df, file_name)
   
    return df



In [16]:
def pega_tweets_filtrado(search_words, until, lang, items, file_name):

    
    api = get_auth_twitter()
    
    cursor = search_for_tweets(api, search_words, until, lang, items)

    tweets_list = iterator_to_list(cursor)

    df = tweets_to_dataframe(tweets_list)
    df = destrinchando_colunas(df)
    df = format_created_at(df)
    df = filter_columns (df)

    create_csv(df, file_name)
    create_pickle(df, file_name)
   
    return df


## Project Name: DisneyPlus

In [19]:
df = pega_tweets_filtrado("DisneyPlus", "", "pt", 15000, 'until_18')

In [20]:
df.created_at.sort_values

<bound method Series.sort_values of 0       2020-11-18 13:26:19
1       2020-11-18 13:26:16
2       2020-11-18 13:26:16
3       2020-11-18 13:26:09
4       2020-11-18 13:26:00
                ...        
14995   2020-11-17 22:46:21
14996   2020-11-17 22:46:18
14997   2020-11-17 22:46:11
14998   2020-11-17 22:46:10
14999   2020-11-17 22:46:10
Name: created_at, Length: 15000, dtype: datetime64[ns]>